In [0]:
# %sql
# DROP TABLE formula1_silver.results;

In [0]:
%run ../configurations/paths_config

In [0]:
%run ../utils/etl_support_functions

In [0]:
dbutils.widgets.text("File Date","2021-03-21")
v_file_date = dbutils.widgets.get("File Date")

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,DateType,StructType,StructField,DoubleType
from pyspark.sql.functions import col,lit,expr
from pyspark.sql import functions as sf
import json
from delta.tables import DeltaTable


In [0]:
new_schema = StructType(fields=[StructField("constructorId",IntegerType(),False),
                                StructField("driverId",IntegerType(),False),
                                StructField("fastestLap",IntegerType(),True),
                                StructField("fastestLapSpeed",DoubleType(),True),
                                StructField("fastestLapTime",StringType(),True),
                                StructField("grid",IntegerType (),True),
                                StructField("laps",IntegerType(),True),
                                StructField("milliseconds",IntegerType(),True),
                                StructField("number",IntegerType(),True),
                                StructField("points",IntegerType(),True),
                                StructField("position",IntegerType(),True),
                                StructField("positionOrder",IntegerType(),True),
                                StructField("positionText",IntegerType(),True),
                                StructField("raceId",IntegerType(),False),
                                StructField("rank",IntegerType(),True),
                                StructField("resultId",IntegerType(),True),
                                StructField("statusId",IntegerType(),True),
                                StructField("time",StringType(),True)
                                ])

In [0]:
raw_df = spark.read.schema(new_schema).json(raw_path+f"/{v_file_date}/results.json") 

In [0]:
renamed_df = raw_df.withColumnRenamed("constructorId","constructor_id").withColumnRenamed("driverId","driver_id").withColumnRenamed("fastestLap","fastest_lap").withColumnRenamed("fastestLapSpeed","fastest_lap_speed").withColumnRenamed("fastestLapTime","fastest_lap_time").withColumnRenamed("positionOrder","position_order").withColumnRenamed("positionText","position_text").withColumnRenamed("raceId","race_id").withColumnRenamed("resultId","result_id")

In [0]:
trimmed_df = renamed_df.drop("statusId")

In [0]:
deDuped_df = trimmed_df.dropDuplicates(["race_id","driver_id"])

In [0]:
audited_df = deDuped_df.withColumn("ingestion_timestamp",sf.current_timestamp())

In [0]:
audited_df = push_partition_col_to_end(audited_df,"race_id")

In [0]:
merge_condition = 'tgt.result_id = upd.result_id  AND tgt.race_id = upd.race_id'
upsert_into_delta_table(audited_df, 'vsarthicat.formula1_silver.results',merge_condition,'race_id')

In [0]:
%sql
(SELECT driver_id,race_id, COUNT(1) 
FROM vsarthicat.formula1_silver.results 
GROUP BY race_id,driver_id
HAVING COUNT(1)>1)